# Latent Dirichlet Allocation (LDA) Topic Modeling

This notebook demonstrates how to do topic modeling using the latent dirichlet allocation method. The following processes are described:

* Importing your [dataset](./key-terms.ipynb#dataset)
* Checking the import was successful with `len()` and `query()`
* Importing libraries including `os`, `warnings`, `gensim`, `nltk`, and `pyLDAvis`
* Writing a helper function to help clean up a single [token](./key-terms.ipynb#token)
* Building a gensim dictionary and training the model
* Computing a topic list
* Visualizing the topic list

This example uses the [`gensim`](https://radimrehurek.com/gensim/index.html) library for building the topic model. A familiarity with gensim is helpful but not required.
____

In [1]:
import os

import warnings
#warnings.filterwarnings('ignore')

In [2]:
import gensim
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import pyLDAvis.gensim

pyLDAvis.enable_notebook()

stop_words = set(stopwords.words('english'))

Initialize a dataset object. 

In [3]:
from tdm_client import Dataset

dset = Dataset('59c090b6-3851-3c65-e016-9181833b4a2c')

Print the text of the query that built this dataset.

In [4]:
dset.query_text()

'All documents from JSTOR published in Shakespeare Quarterly from 1700 - 2019'

Find total number of documents in the dataset using the `len()` function. 

In [5]:
len(dset)

6687

In [6]:
dset.query()

'q=*%3A*&fq=yearPublished%3A%5B1700%20TO%202019%5D&fq=-provider%3Aportico&fq=isPartOf%3A(%22Shakespeare%20Quarterly%22)'

Define a function for processing tokens from the extracted features for volumes in the curated dataset. This function:

* lowercases all tokens
* discards all tokens less than 4 characters
* discards non alphabetical tokens - e.g. --9
* removes stopwords using NLTK's stopword list
* Lemmatizes the token using NLTK's [WordNetLemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html)

In [7]:
def process_token(token):
    token = token.lower()
    if len(token) < 4:
        return
    if not(token.isalpha()):
        return
    if token in stop_words:
        return
    return WordNetLemmatizer().lemmatize(token)

Loop through the volumes in the dataset and make a list of tokens for each volume and then add to a list of the 25 documents in the dataset. We are limiting this example to 25 documents to limit the time it takes to run during demonstrations.

In [8]:
documents = []

for n, unigram_count in enumerate(dset.get_features()):
    this_doc = []
    for token, count in unigram_count.items():
        clean_token = process_token(token)
        if clean_token is None:
            continue
        this_doc += [clean_token] * count
    documents.append(this_doc)
    if n >= 24:
        break
                

Build a gensim dictionary and corpus and then train the model.

In [9]:
num_topics = 10

dictionary = gensim.corpora.Dictionary(documents)

dictionary.filter_extremes(no_below=len(documents) * .10, no_above=0.5)

bow_corpus = [dictionary.doc2bow(doc) for doc in documents]


# train model, this might take some time
model = gensim.models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=15
)


Print the most significant terms, as determined by the model, for each topic.

In [10]:
for topic_num in range(0, num_topics):
    word_ids = model.get_topic_terms(topic_num)
    words = []
    for wid, weight in word_ids:
        word = dictionary.id2token[wid]
        words.append(word)
    print("Topic {}".format(str(topic_num).ljust(5)), " ".join(words))

Topic 0     bassanio shylock merchant money venice shall exchange portia christian love
Topic 1     lear king portrait edgar sexual seemed production male head poor
Topic 2     critical woman male view female question student london comedy dryden
Topic 3     date booth paper manuscript macbeth tragedy effect experience idea find
Topic 4     portrait head state folio picture body painted image drawing copy
Topic 5     henry richard production presented royal principal running used repertory france
Topic 6     lear king edgar form report sense saying nothing meaning worst
Topic 7     troilus folger public quarto version private robert secret back folio
Topic 8     macbeth production line lady appeared seemed almost used problem came
Topic 9     sexual language gloss literary seem although frequently reference already claim


Visualize the model using [`pyLDAvis`](https://pyldavis.readthedocs.io/en/latest/). This visualization takes several minutes to an hour to generate depending on the size of your dataset. To run, remove the `#` symbol on the line below and run the cell. 

In [12]:
pyLDAvis.gensim.prepare(model, bow_corpus, dictionary)

//anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.043478  0.050407       1        1  20.044111
5     -0.070205 -0.125769       2        1  17.670973
6     -0.133188  0.046996       3        1  17.155367
0     -0.073941  0.118594       4        1  16.427608
8     -0.075540 -0.124021       5        1  10.049633
4      0.164791 -0.045154       6        1   7.516266
3      0.007844 -0.007899       7        1   5.165864
7      0.117108 -0.004429       8        1   3.203971
9      0.027159  0.096501       9        1   2.747723
1     -0.007506 -0.005225      10        1   0.018481, topic_info=     Category       Freq        Term      Total  loglift  logprob
1181  Default  61.000000    portrait  61.000000  30.0000  30.0000
1442  Default  48.000000     macbeth  48.000000  29.0000  29.0000
243   Default  79.000000        lear  79.000000  28.0000  28.0000
753   Default  43.000000    bassanio  43.000000  27.0000  27.0000
242   Default  81.000000        king  81.000000  26.0000  26.0000
1018  Default  42.000000     shylock  42.000000  25.0000  25.0000
199   Default  40.000000       edgar  40.000000  24.0000  24.0000
917   Default  37.000000    merchant  37.000000  23.0000  23.0000
1288  Default  43.000000       henry  43.000000  22.0000  22.0000
83    Default  28.000000       money  28.000000  21.0000  21.0000
1109  Default  21.000000        date  21.000000  20.0000  20.0000
290   Default  48.000000     richard  48.000000  19.0000  19.0000
1064  Default  26.000000      venice  26.000000  18.0000  18.0000
274   Default  48.000000  production  48.000000  17.0000  17.0000
214   Default  20.000000       folio  20.000000  16.0000  16.0000
227   Default  19.000000        head  19.000000  15.0000  15.0000
1361  Default  30.000000     tragedy  30.000000  14.0000  14.0000
1248  Default  14.000000       booth  14.000000  13.0000  13.0000
25    Default  29.000000    critical  29.000000  12.0000  12.0000
1178  Default  15.000000     picture  15.000000  11.0000  11.0000
320   Default  30.000000       state  30.000000  10.0000  10.0000
699   Default  20.000000    literary  20.000000   9.0000   9.0000
1518  Default  11.000000     troilus  11.000000   8.0000   8.0000
625   Default  16.000000      public  16.000000   7.0000   7.0000
465   Default  28.000000        male  28.000000   6.0000   6.0000
589   Default  43.000000        form  43.000000   5.0000   5.0000
548   Default  24.000000    although  24.000000   4.0000   4.0000
694   Default  18.000000    language  18.000000   3.0000   3.0000
714   Default  10.000000      sexual  10.000000   2.0000   2.0000
1392  Default  15.000000  manuscript  15.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
242   Topic10   0.001839        king  81.564163  -2.1038  -7.2117
227   Topic10   0.001705        head  19.180573  -0.7320  -7.2874
199   Topic10   0.001754       edgar  40.428875  -1.4493  -7.2590
268   Topic10   0.001689        poor  15.845004  -0.5502  -7.2966
1240  Topic10   0.001616    anything   5.687384   0.4299  -7.3411
299   Topic10   0.001710      seemed  23.705774  -0.9408  -7.2844
1181  Topic10   0.001757    portrait  61.625835  -1.8693  -7.2574
465   Topic10   0.001705        male  28.579300  -1.1306  -7.2871
1568  Topic10   0.001645       gloss   9.949677  -0.1113  -7.3230
151   Topic10   0.001682      almost  24.341082  -0.9837  -7.3008
274   Topic10   0.001708  production  48.729965  -1.6624  -7.2854
320   Topic10   0.001689       state  30.784580  -1.2147  -7.2969
1408  Topic10   0.001667      female  18.546017  -0.7209  -7.3099
254   Topic10   0.001675       moral  24.882021  -1.0099  -7.3050
993   Topic10   0.001677      report  33.117699  -1.2947  -7.3039
753   Topic10   0.001686    bassanio  43.636818  -1.5651  -7.2984
215   Topic10   0.001662        fool  21.356640  -0.8652  -7.3131
296   Topic10   0.001665      saying  24.132505  -0.9856  -7.3113
618   Top